In [3]:
import pandas as pd
import numpy as np
import matplotlib
import seaborn as sb
import matplotlib.pyplot as plt
import matplotlib.cm as cmx
import matplotlib.colors as colors
import math
import pylab
import scipy.stats as stats
import seaborn as sns
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV
from sklearn import metrics
%matplotlib inline

In [4]:
listing = pd.read_csv("./data/listing_processed.csv", header=0)

In [5]:
listing.columns

Index(['id', 'host_id', 'host_since', 'neighbourhood_cleansed', 'zipcode',
       'property_type', 'room_type', 'accommodates', 'bathrooms', 'bedrooms',
       'beds', 'price', 'cleaning_fee', 'guests_included', 'extra_people',
       'minimum_nights', 'availability_30', 'number_of_reviews',
       'cancellation_policy', 'review_cat', 'host_months', 'property_cat'],
      dtype='object')

In [6]:
# print("price over 2000: ", sum(listing.price > 2000))

In [7]:
# STEP 1
# standardize cleaning_fee, number_of_reviews, host_months, extra_people
listing_part1 = listing[['cleaning_fee','number_of_reviews','host_months','extra_people','minimum_nights','availability_30']]
scalar = preprocessing.StandardScaler().fit(listing_part1)
listing_part1 = scalar.transform(listing_part1)


In [8]:
# from sklearn.externals import joblib
# joblib.dump(scalar, "standardize.pkl")
# joblib.load('standardize.pkl').transform(listing_part1)

In [9]:
listing_part1

array([[ 0.60565095,  0.46730886,  2.78059907, -0.58493052, -0.01958909,
        -0.57617341],
       [-0.85422717,  5.87957693,  2.68302077, -0.58493052, -0.29982302,
        -0.41130751],
       [ 1.06666509,  1.08001845,  2.6205268 ,  3.54541962, -0.20641171,
        -0.90590521],
       ...,
       [-0.85422717, -0.51302649,  0.61962346, -0.58493052, -0.20641171,
         1.56708329],
       [-0.85422717, -0.51302649, -1.19599075, -0.58493052, -0.29982302,
         0.99005264],
       [ 5.29262805, -0.51302649, -1.50297866, -0.58493052, -0.01958909,
         0.99005264]])

In [10]:
# replace these 6 columns with new scaled ones

del listing['cleaning_fee']
del listing['number_of_reviews']
del listing['host_months']
del listing['extra_people']
del listing['minimum_nights']
del listing['availability_30']

In [12]:
listing.room_type.unique()

array(['Entire home/apt', 'Private room', 'Shared room', 'Hotel room'],
      dtype=object)

In [203]:
# STEP 2: 
# transfer categorical variables

le1 = preprocessing.LabelEncoder()

temp = pd.DataFrame(listing['neighbourhood_cleansed'].values,
                   columns=['neighbourhood_cleansed'])

temp['temp'] = le1.fit_transform(listing.neighbourhood_cleansed)
temp.drop_duplicates(inplace=True)
temp
temp.to_csv("./saved-models/neighbourhood-le.csv", index=False)



In [193]:
le2 = preprocessing.LabelEncoder()

temp = pd.DataFrame(listing['property_cat'].values,
                   columns=['property_cat'])

temp['temp'] = le2.fit_transform(listing.property_cat)
temp.drop_duplicates(inplace=True)
temp.to_csv("./saved-models/property_cat-le.csv", index=False)

In [16]:
le3 = preprocessing.LabelEncoder()

temp = pd.DataFrame(listing['room_type'].values,
                   columns=['room_type'])

temp['temp'] = le3.fit_transform(listing.room_type)
temp.drop_duplicates(inplace=True)
temp.to_csv("./saved-models/room_type-le.csv", index=False)

In [17]:
le4 = preprocessing.LabelEncoder()

temp = pd.DataFrame(listing['cancellation_policy'].values,
                   columns=['cancellation_policy'])

temp['temp'] = le4.fit_transform(listing.cancellation_policy)
temp.drop_duplicates(inplace=True)
temp.to_csv("./saved-models/cancellation_policy-le.csv", index=False)

In [18]:
le5 = preprocessing.LabelEncoder()

temp = pd.DataFrame(listing['review_cat'].values,
                   columns=['review_cat'])

temp['temp'] = le5.fit_transform(listing.review_cat)
temp.drop_duplicates(inplace=True)
temp.to_csv("./saved-models/review_cat-le.csv", index=False)

In [131]:
from sklearn.externals import joblib

joblib.dump(le1, "neighbourhood.pkl")



['neighbourhood.pkl']

In [83]:
# joblib.dump(le1, "neighbourhood.pkl")
# joblib.dump(le2, "property_cat.pkl")
# joblib.dump(le3, "room_type.pkl")
# joblib.dump(le4, "cancellation_policy.pkl")
# joblib.dump(le5, "review_cat.pkl")

In [89]:
listing

,id,host_id,host_since,neighbourhood_cleansed,zipcode,property_type,room_type,accommodates,bathrooms,bedrooms,beds,price,guests_included,cancellation_policy,review_cat,property_cat
0,2595,2845,9/9/08,129,10018,Apartment,0,2,1.0,0,1,225,1,3,2,0
1,3831,4869,12/7/08,40,11238,Guest suite,0,3,1.0,1,4,89,1,1,0,2
2,5099,7322,2/2/09,139,10016,Apartment,0,2,1.0,1,1,200,2,1,0,0
3,5178,8967,3/3/09,97,10019,Apartment,2,2,1.0,1,1,79,1,3,0,0
4,5203,7490,2/5/09,204,10025,Apartment,2,1,1.0,1,1,79,1,0,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50191,42890680,258247777,4/26/19,96,10030,Apartment,2,2,1.0,1,1,150,1,0,3,0
50192,42890730,79065824,6/21/16,204,10019,Apartment,0,2,1.0,0,0,149,1,0,3,0
50193,42891018,11849741,2/1/14,211,10014,Apartment,0,2,1.0,0,1,188,1,0,3,0
50194,42891637,209711461,8/15/18,51,11225,Apartment,2,2,1.0,1,1,1100,1,0,3,0


In [90]:
zipcodes = listing.zipcode.copy()

In [91]:
data_Y = listing['price']

# those columns will not be fed into models
data_X = listing.drop(['id', 'host_id', 'zipcode','host_since', 'price','property_type'], axis=1)

In [92]:
data_X.columns

Index(['neighbourhood_cleansed', 'room_type', 'accommodates', 'bathrooms',
       'bedrooms', 'beds', 'guests_included', 'cancellation_policy',
       'review_cat', 'property_cat'],
      dtype='object')

In [67]:
# train change to dummy:property_type，room_type，neighbourhood_cleansed，cancellation_policy，review_cat

dummyCols=pd.get_dummies(data_X['room_type'])
dummyCols.columns = [str(col) + 'room_type' for col in dummyCols.columns]
data_X=data_X.join(dummyCols)
del data_X['room_type']

dummyCols2=pd.get_dummies(data_X['neighbourhood_cleansed'])
dummyCols2.columns = [str(col) + 'neighbourhood_cleansed' for col in dummyCols2.columns]
data_X=data_X.join(dummyCols2)
del data_X['neighbourhood_cleansed']


dummyCols3=pd.get_dummies(data_X['cancellation_policy'])
dummyCols3.columns = [str(col) + 'cancellation_policy' for col in dummyCols3.columns]
data_X=data_X.join(dummyCols3)
del data_X['cancellation_policy']

dummyCols4=pd.get_dummies(data_X['review_cat'])
dummyCols4.columns = [str(col) + 'review_cat' for col in dummyCols4.columns]
data_X=data_X.join(dummyCols4)
del data_X['review_cat']

dummyCols5=pd.get_dummies(data_X['property_cat'])
dummyCols5.columns = [str(col) + 'property_cat' for col in dummyCols5.columns]
data_X=data_X.join(dummyCols5)
del data_X['property_cat']

In [95]:
data_X = np.concatenate((data_X, listing_part1), axis=1)

In [87]:
# data_X: 

array([[ 0.60565095,  0.46730886,  2.78059907, -0.58493052, -0.01958909,
        -0.57617341],
       [-0.85422717,  5.87957693,  2.68302077, -0.58493052, -0.29982302,
        -0.41130751],
       [ 1.06666509,  1.08001845,  2.6205268 ,  3.54541962, -0.20641171,
        -0.90590521],
       ...,
       [-0.85422717, -0.51302649,  0.61962346, -0.58493052, -0.20641171,
         1.56708329],
       [-0.85422717, -0.51302649, -1.19599075, -0.58493052, -0.29982302,
         0.99005264],
       [ 5.29262805, -0.51302649, -1.50297866, -0.58493052, -0.01958909,
         0.99005264]])

In [69]:
data_X = data_X[(data_Y < 2000) & (data_Y > 0)]
data_Y = data_Y[(data_Y < 2000) & (data_Y > 0)]

In [70]:
train = data_X.shape[0] * 0.75

In [98]:
len(data_X[0])

16

In [71]:
train_X = data_X[:int(train)]
test_X = data_X[int(train):]
train_y = data_Y[:int(train)]
test_y = data_Y[int(train):]

# train_X = train_X[ train_y < 2000 and ]
# train_X = train_X[ train_y > 0]
# train_y = train_y[ train_y < 2000]
# train_y = train_y[ train_y > 0]

# test_X = test_X[test_y > 0]
# test_y = test_y[test_y > 0]

# zipcodes_test = zipcodes[int(train):]
# zipcodes_test = zipcodes_test[test_y > 0]

In [72]:
train_X.shape

(37511, 29)

In [73]:
# build model
# Linear Regression model

model_1=LinearRegression()
model_1.fit(train_X,train_y)
y_pred = model_1.predict(test_X)


In [74]:
print('Mean Absolute Error:', metrics.mean_absolute_error(test_y, y_pred))  
print('Mean Squared Error:', metrics.mean_squared_error(test_y, y_pred))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(test_y, y_pred)))

Mean Absolute Error: 70.79731267244482
Mean Squared Error: 17950.255014026614
Root Mean Squared Error: 133.97856177025716


In [75]:
train_X

array([[ 2.        ,  1.        ,  0.        , ..., -0.58493052,
        -0.01958909, -0.57617341],
       [ 3.        ,  1.        ,  1.        , ..., -0.58493052,
        -0.29982302, -0.41130751],
       [ 2.        ,  1.        ,  1.        , ...,  3.54541962,
        -0.20641171, -0.90590521],
       ...,
       [ 3.        ,  1.        ,  1.        , ..., -0.58493052,
         1.0079353 ,  1.56708329],
       [ 1.        ,  1.        ,  1.        , ..., -0.58493052,
        -0.25311737, -0.90590521],
       [ 2.        ,  1.        ,  1.        , ..., -0.58493052,
        -0.1130004 , -0.90590521]])

In [76]:
train_y_log = np.log(train_y+0.000001)
ridge = RidgeCV(alphas=[1e-2, 1e-1, 1]).fit(train_X, train_y_log)
y_pred_ridge = ridge.predict(test_X)

In [78]:
# joblib.dump(ridge, "ridge.pkl")

In [79]:
ridge.score(train_X,train_y_log)

0.5326674439926598

In [80]:
ridge.score(test_X, np.log(test_y + 0.000001))

0.569048592313127

In [81]:
print('Mean Squared Error:', metrics.mean_squared_error(test_y, np.exp(y_pred_ridge))) 
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(test_y, np.exp(y_pred_ridge))))

Mean Squared Error: 19288.014137115988
Root Mean Squared Error: 138.8812951304674


In [230]:
print('Mean Squared Error:', metrics.mean_squared_error(y_test_low, np.exp(y_pred_low))) 
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test_low, np.exp(y_pred_low))))

Mean Squared Error: 16189.332833265333
Root Mean Squared Error: 127.23730912458552


In [0]:
# plt.hist(np.log(train_y+0.0000001),bins = 200)

In [49]:
# train_y_log = np.log(train_y+0.000001)
Lasso = LassoCV(alphas=[1e-2, 1e-1, 1]).fit(train_X, train_y_log)
y_pred_lasso = Lasso.predict(test_X)

In [50]:
Lasso.score(train_X,train_y_log)

0.5179492858301653

In [51]:
Lasso.score(test_X, np.log(test_y + 0.000001))

0.5428618914718288

In [52]:
print('Mean Squared Error:', metrics.mean_squared_error(test_y[test_y < 2000], np.exp(y_pred_lasso[test_y < 2000]))) 
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(test_y[test_y < 2000], np.exp(y_pred_lasso[test_y < 2000]))))

Mean Squared Error: 18820.624911571795
Root Mean Squared Error: 137.18828270509036


In [66]:
print(sum(Lasso.coef_!=0))
print(len(Lasso.coef_))
print(len(train_X.columns))

20
253
253


In [67]:
Vars = train_X.columns[Lasso.coef_!=0]

In [68]:
Vars

Index(['Unnamed: 0', 'accommodates', 'bathrooms', 'bedrooms', 'cleaning_fee',
       'extra_people', 'minimum_nights', 'availability_30',
       'number_of_reviews', 'host_months', '0room_type', '2room_type',
       '12neighbourhood_cleansed', '27neighbourhood_cleansed',
       '97neighbourhood_cleansed', '129neighbourhood_cleansed', '0review_cat',
       '3review_cat', '1property_cat', '2property_cat'],
      dtype='object')

In [0]:
# Model 2
# KNN

In [69]:
from sklearn.neighbors import KNeighborsRegressor

knn = KNeighborsRegressor(n_neighbors=10, metric='euclidean')
knn.fit(train_X[Vars], train_y)
y_pred = knn.predict(test_X[Vars])


In [70]:
#print("Accuracy:", metrics.accuracy_score(test_y, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(test_y[test_y < 2000], y_pred[test_y < 2000])))

Root Mean Squared Error: 184.899063813201


In [76]:
# Model 3
# PCA

In [220]:
from sklearn.decomposition import PCA
pca = PCA(n_components=20)
pca.fit(train_X)
train_x_pca = pca.transform(train_X)
test_x_pca = pca.transform(test_X)

In [221]:
test_x_pca.shape

(12618, 20)

In [53]:
model_1_pca=LinearRegression()
model_1_pca.fit(train_x_pca,train_y_log)
y_pred_pca = model_1_pca.predict(test_x_pca)

In [83]:
model_1_pca.score(train_x_pca,train_y_log)

0.5112658310352178

In [81]:
print('Mean Absolute Error:', metrics.mean_absolute_error(test_y, np.exp(y_pred_pca)))  
print('Mean Squared Error:', metrics.mean_squared_error(test_y, np.exp(y_pred_pca)))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(test_y, np.exp(y_pred_pca))))

Mean Absolute Error: 113.72369308983492
Mean Squared Error: 472111.46430226264
Root Mean Squared Error: 687.103677986272


In [ ]:
plt.plot(s)

In [101]:
sales = pd.read_csv("zip_Data.csv")

In [103]:
sales.Price = sales.Price / 36.83

In [104]:
sales

,RegionName,Price
0,10025,30416.146267
1,10023,33043.795819
2,10002,29823.106163
3,11226,17416.978911
4,10467,17416.978911
5,10016,21317.241379
6,11201,26042.474431
7,11235,10589.410806
8,11375,11050.303196
9,10009,23473.418409


In [68]:
len(sales.RegionName.unique())

176

In [69]:
len(set(zipcodes_test))

209

In [106]:
pred_price_by_zip = pd.DataFrame({"zip":zipcodes_test, "lasso_pred_price": np.exp(y_pred_lasso), "ridge_pred_price": np.exp(y_pred_ridge), "knn_pred_price": y_pred})

In [107]:
pred_price_by_zip

,zip,lasso_pred_price,ridge_pred_price,knn_pred_price
37987,10454,195.988387,119.342366,178.815430
37988,10016,228.271837,267.555882,312.465820
37989,11220,58.956774,49.271334,44.864258
37990,11203,191.748204,173.631577,272.821289
37991,10028,155.093004,182.527104,192.964844
37992,11206,147.014236,162.665765,188.779297
37993,10009,204.895237,259.456066,267.955078
37994,11211,158.145878,147.542439,164.164062
37995,11206,161.334390,168.084909,194.145508
37996,11207,68.337345,67.982715,128.209961


In [0]:
# all variables 677.381341
# TODO:
# lasso variable selection
# price prediction tool - what features would be beneficial
# price vs rental(not up to date)
# combine models
# our idea: 